In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Настройки для графиков
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 8)

# Загрузка данных
try:
    df = pd.read_parquet('transaction_fraud_data.parquet')
except FileNotFoundError:
    print("Файл 'transaction_fraud_data.parquet' не найден.")
    df = pd.DataFrame()

if not df.empty:
    # 1. Фильтруем данные, оставляя только мошеннические транзакции
    fraud_df = df[df['is_fraud'] == True].copy()

    # 2. Находим топ-10 клиентов по числу фрод-операций
    customer_fraud_activity = fraud_df.groupby('customer_id').agg(
        fraud_transactions_count=('transaction_id', 'count'),
        total_fraud_amount=('amount', 'sum')
    ).reset_index()
    top_fraud_customers = customer_fraud_activity.sort_values('fraud_transactions_count', ascending=False).head(10)
    
    # 3. Считаем глобальные метрики для расчета процентов
    total_fraud_transactions_all = len(fraud_df)
    total_transaction_amount_all = df['amount'].sum()
    
    # 4. Вычисляем совокупный вклад топ-10 клиентов
    sum_fraud_count_top_10 = top_fraud_customers['fraud_transactions_count'].sum()
    sum_fraud_amount_top_10 = top_fraud_customers['total_fraud_amount'].sum()
    
    # Расчет процентов
    percentage_of_all_fraud_txns = (sum_fraud_count_top_10 / total_fraud_transactions_all) * 100
    percentage_of_all_amount = (sum_fraud_amount_top_10 / total_transaction_amount_all) * 100

    # 5. Функция для отрисовки графика
    def plot_top_fraud_customers(analysis_df):
        """Строит двухосевую диаграмму для топ-10 мошеннических аккаунтов."""
        fig, ax1 = plt.subplots()

        # Столбчатая диаграмма: Количество мошеннических операций (левая ось)
        sns.barplot(x='customer_id', y='fraud_transactions_count', data=analysis_df, ax=ax1, color='orange', alpha=0.8)
        ax1.set_ylabel('Количество мошеннических операций', color='orange', fontsize=12)
        ax1.tick_params(axis='y', labelcolor='orange')
        ax1.tick_params(axis='x', rotation=45)
        plt.setp(ax1.get_xticklabels(), ha='right')

        # Вторая ось Y для суммы
        ax2 = ax1.twinx()

        # Линейный график: Общая сумма мошенничества (правая ось)
        sns.lineplot(x='customer_id', y='total_fraud_amount', data=analysis_df, ax=ax2, color='darkred', marker='o', sort=False, lw=2)
        ax2.set_ylabel('Общая сумма ущерба (amount)', color='darkred', fontsize=12)
        ax2.tick_params(axis='y', labelcolor='darkred')

        plt.title('Топ-10 "Крупных игроков": Анализ количества и суммы мошеннических операций', fontsize=16, pad=20)
        fig.tight_layout()
        plt.show()

    # Отрисовка графика
    plot_top_fraud_customers(top_fraud_customers)
    
    # Вывод итоговой текстовой информации
    print("\n" + "="*50)
    print("--- АНАЛИЗ ВЛИЯНИЯ ТОП-10 МОШЕННИЧЕСКИХ АККАУНТОВ ---")
    print("="*50)
    print(f"Всего 10 аккаунтов ответственны за {sum_fraud_count_top_10} мошеннических операций.")
    print(f"Это составляет: {percentage_of_all_fraud_txns:.2f}% от ВСЕХ мошеннических операций в датасете.")
    print("-" * 50)
    print(f"Общая сумма ущерба от этих 10 аккаунтов: {sum_fraud_amount_top_10:,.2f}.")
    print(f"Это составляет: {percentage_of_all_amount:.4f}% от ОБЩЕЙ СУММЫ ВСЕХ транзакций.")
    print("="*50)

In [ ]:
if not df.empty:
    # Используем тот же отфильтрованный fraud_df
    
    # Группируем по отпечатку устройства
    device_fraud_network = fraud_df.groupby('device_fingerprint').agg(
        fraud_transactions_count=('transaction_id', 'count'),
        unique_customers_involved=('customer_id', 'nunique') # Считаем уникальных клиентов
    ).reset_index()

    # Нас интересуют устройства, которые засветились более чем у одного клиента
    fraud_rings_by_device = device_fraud_network[device_fraud_network['unique_customers_involved'] > 1]

    # Сортируем по количеству вовлеченных клиентов
    top_fraud_devices = fraud_rings_by_device.sort_values('unique_customers_involved', ascending=False)
    
    print("\n--- Топ-10 устройств, используемых в мошеннических сетях ---")
    print(top_fraud_devices.head(10))

In [ ]:
if not df.empty:
    # Группируем по IP-адресу
    ip_fraud_network = fraud_df.groupby('ip_address').agg(
        fraud_transactions_count=('transaction_id', 'count'),
        unique_customers_involved=('customer_id', 'nunique')
    ).reset_index()

    # Фильтруем IP, которые использовались более чем одним клиентом
    fraud_rings_by_ip = ip_fraud_network[ip_fraud_network['unique_customers_involved'] > 1]
    
    # Сортируем
    top_fraud_ips = fraud_rings_by_ip.sort_values('unique_customers_involved', ascending=False)
    
    print("\n--- Топ-10 IP-адресов, используемых в мошеннических сетях ---")
    print(top_fraud_ips.head(10))